In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize

# Пример текста
text = "Привет, мир! Как дела? Я изучаю ИИ и обработку естественного языка."

# Токенизация по предложениям
sentences = sent_tokenize(text)
print("Предложения:", sentences)

# Токенизация по словам
words = word_tokenize(text)
print("Слова:", words)

# Простая токенизация по пробелам
simple_tokens = text.split()
print("Простые токены:", simple_tokens)

In [ ]:
import spacy
import re
from nltk.tokenize import RegexpTokenizer

# Загрузка модели русского языка
nlp = spacy.load("ru_core_news_sm")

text = "Изучение нейронных сетей (CNN, RNN) является важным для понимания современных моделей ИИ, таких как GPT-3."

# Токенизация с помощью spaCy
doc = nlp(text)
spacy_tokens = [token.text for token in doc]
print("SpaCy токены:", spacy_tokens)

# Токенизация с помощью регулярных выражений
pattern = r'\w+|[^\w\s]'
regexp_tokenizer = RegexpTokenizer(pattern)
regex_tokens = regexp_tokenizer.tokenize(text)
print("Токены по регулярным выражениям:", regex_tokens)

# Извлечение только определенных токенов (например, существительных)
nouns = [token.text for token in doc if token.pos_ == "NOUN"]
print("Существительные:", nouns)

In [ ]:
import re
from collections import Counter

class SimpleBPE:
    def __init__(self, vocab_size=10000):
        self.vocab_size = vocab_size
        self.vocab = {}
        self.inverse_vocab = {}
        self.merges = {}
        
    def train(self, texts, initial_vocab=None):
        # Инициализация словаря символами
        if initial_vocab is None:
            chars = set()
            for text in texts:
                chars.update(text)
            self.vocab = {c: i for i, c in enumerate(sorted(chars))}
            
        # Токенизация текстов на символы
        word_counts = Counter()
        for text in texts:
            words = text.split()
            for word in words:
                word_counts[' '.join(list(word)) + ' </w>'] += 1
                
        # Обучение BPE
        vocab_size = len(self.vocab)
        while vocab_size < self.vocab_size:
            pairs = self._get_stats(word_counts)
            if not pairs:
                break
                
            best_pair = max(pairs, key=pairs.get)
            new_token = ''.join(best_pair)
            self.merges[best_pair] = new_token
            self.vocab[new_token] = vocab_size
            vocab_size += 1
            
            # Обновление корпуса
            new_word_counts = Counter()
            for word, count in word_counts.items():
                new_word = self._merge_word(word, best_pair)
                new_word_counts[new_word] += count
            word_counts = new_word_counts
            
        self.inverse_vocab = {i: token for token, i in self.vocab.items()}
        
    def _get_stats(self, word_counts):
        pairs = Counter()
        for word, freq in word_counts.items():
            symbols = word.split()
            for i in range(len(symbols) - 1):
                pairs[(symbols[i], symbols[i+1])] += freq
        return pairs
    
    def _merge_word(self, word, pair):
        pair_str = ' '.join(pair)
        pair_repl = ''.join(pair)
        parts = word.split()
        i = 0
        while i < len(parts) - 1:
            if parts[i] == pair[0] and parts[i+1] == pair[1]:
                parts[i] = pair_repl
                parts.pop(i+1)
            else:
                i += 1
        return ' '.join(parts)
    
    def tokenize(self, text):
        words = text.split()
        result = []
        for word in words:
            word = ' '.join(list(word)) + ' </w>'
            while True:
                pairs = self._find_pairs(word)
                if not pairs:
                    break
                word = self._apply_merges(word, pairs)
            result.extend(word.split())
        return result
    
    def _find_pairs(self, word):
        parts = word.split()
        pairs = []
        for i in range(len(parts) - 1):
            pair = (parts[i], parts[i+1])
            if pair in self.merges:
                pairs.append(pair)
        return pairs
    
    def _apply_merges(self, word, pairs):
        if not pairs:
            return word
        pair = pairs[0]
        pair_str = ' '.join(pair)
        pair_repl = ''.join(pair)
        return word.replace(pair_str, pair_repl)

# Пример использования
texts = ["Привет мир", "Изучаю нейронные сети", "Обработка естественного языка"]
bpe = SimpleBPE(vocab_size=50)
bpe.train(texts)
tokens = bpe.tokenize("Привет модели нейронных сетей")
print("BPE токены:", tokens)

In [ ]:
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# Инициализация лемматизатора
lemmatizer = WordNetLemmatizer()

# Пример текста (на английском, так как WordNet для английского)
text = "The children were playing games. They are running in the park."
words = word_tokenize(text)

# Лемматизация
lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
print("Исходные слова:", words)
print("Лемматизированные слова:", lemmatized_words)

# Лемматизация с указанием части речи
lemmatized_verbs = [lemmatizer.lemmatize(word, pos='v') for word in words]
print("Лемматизированные глаголы:", lemmatized_verbs)

In [ ]:
import pymorphy2
from nltk.tokenize import word_tokenize

# Инициализация морфологического анализатора
morph = pymorphy2.MorphAnalyzer()

# Пример текста
text = "Студенты изучали новые алгоритмы машинного обучения. Профессора рассказывали о нейронных сетях."
words = word_tokenize(text)

# Функция для лемматизации
def lemmatize_text(tokens):
    return [morph.parse(token)[0].normal_form for token in tokens]

lemmatized_words = lemmatize_text(words)
print("Исходные слова:", words)
print("Лемматизированные слова:", lemmatized_words)

# Анализ частей речи
pos_tags = [(word, morph.parse(word)[0].tag.POS) for word in words]
print("Части речи:", pos_tags)

# Фильтрация по части речи
nouns = [word for word in words if morph.parse(word)[0].tag.POS == 'NOUN']
print("Существительные:", nouns)

In [ ]:
import re
import numpy as np
from collections import defaultdict
import pickle

class RuleLemmatizer:
    def __init__(self):
        self.suffix_rules = {}  # правила замены суффиксов
        self.exceptions = {}    # исключения
        self.cache = {}         # кэш для ускорения работы
        
    def add_rule(self, suffix, replacement, pos=None):
        """Добавляет правило замены суффикса"""
        if pos not in self.suffix_rules:
            self.suffix_rules[pos] = []
        self.suffix_rules[pos].append((suffix, replacement))
        
    def add_exception(self, word, lemma, pos=None):
        """Добавляет исключение"""
        self.exceptions[(word, pos)] = lemma
        
    def train(self, word_lemma_pairs, pos_tags=None):
        """Обучение на парах слово-лемма"""
        if pos_tags is None:
            pos_tags = [None] * len(word_lemma_pairs)
            
        # Сбор статистики по суффиксам
        suffix_stats = defaultdict(lambda: defaultdict(int))
        for (word, lemma), pos in zip(word_lemma_pairs, pos_tags):
            # Пропускаем слова без изменений
            if word == lemma:
                continue
                
            # Найдем общий префикс
            i = 0
            while i < min(len(word), len(lemma)) and word[i] == lemma[i]:
                i += 1
                
            word_suffix = word[i:]
            lemma_suffix = lemma[i:]
            
            suffix_stats[(pos, word_suffix)][lemma_suffix] += 1
            
            # Добавляем исключения для коротких слов
            if len(word) < 5:
                self.add_exception(word, lemma, pos)
                
        # Создаем правила на основе статистики
        for (pos, word_suffix), replacements in suffix_stats.items():
            # Выбираем наиболее частую замену
            best_replacement = max(replacements.items(), key=lambda x: x[1])[0]
            self.add_rule(word_suffix, best_replacement, pos)
            
    def lemmatize(self, word, pos=None):
        """Лемматизирует слово с учетом части речи"""
        # Проверяем кэш
        cache_key = (word, pos)
        if cache_key in self.cache:
            return self.cache[cache_key]
            
        # Проверяем исключения
        if (word, pos) in self.exceptions:
            self.cache[cache_key] = self.exceptions[(word, pos)]
            return self.exceptions[(word, pos)]
            
        # Применяем правила
        if pos in self.suffix_rules:
            for suffix, replacement in self.suffix_rules[pos]:
                if word.endswith(suffix):
                    lemma = word[:-len(suffix)] + replacement
                    self.cache[cache_key] = lemma
                    return lemma
                    
        # Если нет правил для данной части речи, проверяем общие правила
        if None in self.suffix_rules:
            for suffix, replacement in self.suffix_rules[None]:
                if word.endswith(suffix):
                    lemma = word[:-len(suffix)] + replacement
                    self.cache[cache_key] = lemma
                    return lemma
                    
        # Если не нашли правил, возвращаем слово без изменений
        self.cache[cache_key] = word
        return word
        
    def save(self, filepath):
        """Сохраняет модель в файл"""
        with open(filepath, 'wb') as f:
            pickle.dump((self.suffix_rules, self.exceptions), f)
            
    def load(self, filepath):
        """Загружает модель из файла"""
        with open(filepath, 'rb') as f:
            self.suffix_rules, self.exceptions = pickle.load(f)
        self.cache = {}  # Сбрасываем кэш

# Пример использования
# Создаем обучающий набор (упрощенный для примера)
training_data = [
    (("студенты", "студент"), "NOUN"),
    (("изучали", "изучать"), "VERB"),
    (("новые", "новый"), "ADJ"),
    (("алгоритмы", "алгоритм"), "NOUN"),
    (("профессора", "профессор"), "NOUN"),
    (("рассказывали", "рассказывать"), "VERB")
]

word_lemma_pairs = [pair[0] for pair in training_data]
pos_tags = [pair[1] for pair in training_data]

# Создаем и обучаем лемматизатор
lemmatizer = RuleLemmatizer()
lemmatizer.train(word_lemma_pairs, pos_tags)

# Тестируем на новых словах
test_words = ["алгоритмов", "изучающие", "рассказывает", "студентам"]
for word in test_words:
    print(f"{word} -> {lemmatizer.lemmatize(word)}")

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import numpy as np

# Пример корпуса текстов
corpus = [
    "Нейронные сети революционизируют искусственный интеллект",
    "Глубокое обучение - подраздел машинного обучения",
    "Трансформеры используются для обработки естественного языка",
    "Нейронные сети и глубокое обучение - основа современного ИИ"
]

# Векторизация с использованием Bag of Words
bow_vectorizer = CountVectorizer()
bow_matrix = bow_vectorizer.fit_transform(corpus)

print("BoW словарь:", bow_vectorizer.get_feature_names_out())
print("BoW матрица формы:", bow_matrix.shape)
print("BoW первого документа:\n", bow_matrix[0].toarray())

# Векторизация с использованием TF-IDF
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)

print("\nTF-IDF словарь:", tfidf_vectorizer.get_feature_names_out())
print("TF-IDF матрица формы:", tfidf_matrix.shape)
print("TF-IDF первого документа:\n", tfidf_matrix[0].toarray())

# Сравнение документов на основе косинусного сходства
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(tfidf_matrix)
print("\nМатрица сходства документов:\n", similarity_matrix)

In [ ]:
import gensim
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# Подготовка данных
corpus = [
    "Нейронные сети революционизируют искусственный интеллект",
    "Глубокое обучение - подраздел машинного обучения",
    "Трансформеры используются для обработки естественного языка",
    "Нейронные сети и глубокое обучение - основа современного ИИ",
    "Модели BERT применяются для анализа текстов",
    "GPT может генерировать тексты на естественном языке",
    "Векторное представление слов важно для понимания текста"
]

# Токенизация
tokenized_corpus = [word_tokenize(sentence.lower()) for sentence in corpus]

# Обучение Word2Vec модели
w2v_model = Word2Vec(sentences=tokenized_corpus, vector_size=100, window=5, min_count=1, workers=4)

# Исследование векторов
print("Размерность векторов:", w2v_model.wv.vector_size)
print("Словарь:", list(w2v_model.wv.index_to_key))

# Нахождение ближайших слов
similar_words = w2v_model.wv.most_similar("нейронные", topn=3)
print("Слова, похожие на 'нейронные':", similar_words)

# Визуализация векторов с помощью PCA
def plot_word_vectors(model, words):
    word_vectors = [model.wv[word] for word in words]
    pca = PCA(n_components=2)
    result = pca.fit_transform(word_vectors)
    
    plt.figure(figsize=(10, 8))
    plt.scatter(result[:, 0], result[:, 1], c='lightblue')
    
    for i, word in enumerate(words):
        plt.annotate(word, xy=(result[i, 0], result[i, 1]))
    
    plt.title("2D проекция векторов слов")
    plt.show()

# Выбираем несколько интересных слов для визуализации
words_to_plot = ['нейронные', 'сети', 'глубокое', 'обучение', 'трансформеры', 'bert', 'gpt', 'текста']
words_in_vocab = [word for word in words_to_plot if word in w2v_model.wv]
plot_word_vectors(w2v_model, words_in_vocab)

# Использование предобученных векторов
from gensim.models import KeyedVectors

# Функция загрузки предобученных векторов (для примера)
def load_pretrained_vectors():
    # В реальности нужно скачать модель, например, с RusVectōrēs
    # Пример для демонстрации:
    model = gensim.models.KeyedVectors.load_word2vec_format('path_to_vectors.bin', binary=True)
    return model

# Функция для анализа с предобученными векторами
def analyze_with_pretrained(text, model):
    tokens = word_tokenize(text.lower())
    valid_tokens = [word for word in tokens if word in model.key_to_index]
    if not valid_tokens:
        return None
    
    # Среднее векторов слов как вектор предложения 
    sentence_vector = sum(model[word] for word in valid_tokens) / len(valid_tokens)
    return sentence_vector

In [ ]:
import torch
from transformers import BertTokenizer, BertModel
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

# Загрузка предобученного BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained('bert-base-multilingual-cased')

# Функция для получения BERT-эмбеддингов для текста
def get_bert_embeddings(text, model, tokenizer, pooling='mean'):
    # Токенизация
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    
    # Получение эмбеддингов
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Hidden states - все слои модели
    hidden_states = outputs.last_hidden_state
    
    if pooling == 'cls':
        # Используем только [CLS] токен
        return hidden_states[:, 0, :].numpy()
    elif pooling == 'mean':
        # Среднее по всем токенам
        # Создаем маску для игнорирования padding токенов
        attention_mask = inputs['attention_mask']
        mask = attention_mask.unsqueeze(-1).expand(hidden_states.size()).float()
        masked_embeddings = hidden_states * mask
        summed = torch.sum(masked_embeddings, 1)
        counts = torch.clamp(torch.sum(mask, 1), min=1e-9)
        mean_pooled = summed / counts
        return mean_pooled.numpy()
    else:
        raise ValueError("Поддерживаются только 'cls' и 'mean' стратегии пулинга")

# Пример текстов
sentences = [
    "Нейронные сети революционизируют ИИ",
    "Глубокое обучение - важный подраздел ИИ",
    "BERT используется для анализа текстов",
    "Трансформеры отлично работают с последовательностями",
    "Векторы слов позволяют понять семантику"
]

# Получаем эмбеддинги для всех предложений
embeddings = np.vstack([get_bert_embeddings(sentence, model, tokenizer) for sentence in sentences])

# Визуализация с помощью t-SNE
tsne = TSNE(n_components=2, random_state=42)
reduced_embeddings = tsne.fit_transform(embeddings)

plt.figure(figsize=(10, 6))
plt.scatter(reduced_embeddings[:, 0], reduced_embeddings[:, 1], c='blue')

for i, sentence in enumerate(sentences):
    plt.annotate(sentence, (reduced_embeddings[i, 0], reduced_embeddings[i, 1]))

plt.title('t-SNE визуализация BERT эмбеддингов предложений')
plt.tight_layout()
plt.show()

# Функция для сравнения семантической близости предложений
def compute_similarity(embedding1, embedding2):
    # Косинусное сходство
    similarity = np.dot(embedding1, embedding2) / (np.linalg.norm(embedding1) * np.linalg.norm(embedding2))
    return similarity

# Матрица сходства всех предложений
similarity_matrix = np.zeros((len(sentences), len(sentences)))
for i in range(len(sentences)):
    for j in range(len(sentences)):
        similarity_matrix[i, j] = compute_similarity(embeddings[i][0], embeddings[j][0])

print("Матрица сходства предложений:")
print(similarity_matrix)

# Пример задачи классификации текста с использованием BERT-эмбеддингов
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Предположим, у нас есть эмбеддинги и метки
X = embeddings.reshape(len(sentences), -1)  # Разворачиваем 3D в 2D
y = np.array([0, 0, 1, 1, 1])  # Пример меток классов

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучение простого классификатора
clf = LogisticRegression(random_state=42)
clf.fit(X_train, y_train)

# Оценка качества
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Пример данных (тексты и их категории)
texts = [
    "Нейронные сети обеспечивают высокую точность распознавания образов",
    "Глубокое обучение произвело революцию в компьютерном зрении",
    "Трансформеры показывают отличные результаты в обработке языка",
    "Экономика России показывает признаки восстановления",
    "Инфляция влияет на покупательную способность населения",
    "Центральный банк регулирует финансовую систему страны",
    "Футбольная команда выиграла важный матч сезона",
   "Спортсмены готовятся к чемпионату мира по легкой атлетике",
   "Олимпийские игры собирают лучших спортсменов со всего мира"
]

categories = ["ИИ", "ИИ", "ИИ", "Экономика", "Экономика", "Экономика", "Спорт", "Спорт", "Спорт"]

# Создаем датафрейм
df = pd.DataFrame({"text": texts, "category": categories})

# Разделяем на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
   df["text"], df["category"], test_size=0.3, random_state=42
)

# Векторизация текстов с помощью BoW
vectorizer = CountVectorizer()
X_train_bow = vectorizer.fit_transform(X_train)
X_test_bow = vectorizer.transform(X_test)

# Обучение Naive Bayes классификатора
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_bow, y_train)

# Предсказание и оценка результатов
y_pred = nb_classifier.predict(X_test_bow)
accuracy = accuracy_score(y_test, y_pred)
print(f"Точность классификации: {accuracy:.2f}")
print("\nОтчет по классификации:")
print(classification_report(y_test, y_pred))

# Визуализация матрицы ошибок
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", 
           xticklabels=nb_classifier.classes_, 
           yticklabels=nb_classifier.classes_)
plt.xlabel("Предсказанная категория")
plt.ylabel("Истинная категория")
plt.title("Матрица ошибок")
plt.tight_layout()
plt.show()

# Проверка классификации новых текстов
new_texts = [
   "Искусственный интеллект применяется в самых разных областях",
   "Рост ВВП превысил прогнозы аналитиков",
   "Чемпионат мира по хоккею пройдет в следующем году"
]

new_texts_bow = vectorizer.transform(new_texts)
predictions = nb_classifier.predict(new_texts_bow)
probabilities = nb_classifier.predict_proba(new_texts_bow)

for i, text in enumerate(new_texts):
   print(f"\nТекст: {text}")
   print(f"Предсказанная категория: {predictions[i]}")
   print("Вероятности по классам:")
   for j, cls in enumerate(nb_classifier.classes_):
       print(f"  {cls}: {probabilities[i, j]:.2f}")

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Создаем более обширный набор данных
texts = [
    "Нейронные сети обеспечивают высокую точность распознавания образов",
    "Глубокое обучение произвело революцию в компьютерном зрении",
    "Трансформеры показывают отличные результаты в обработке языка",
    "Алгоритмы машинного обучения применяются для прогнозирования",
    "Искусственный интеллект решает сложные задачи оптимизации",
    "Экономика России показывает признаки восстановления",
    "Инфляция влияет на покупательную способность населения",
    "Центральный банк регулирует финансовую систему страны",
    "Анализ экономических показателей помогает принимать решения",
    "Рынок недвижимости переживает период роста цен",
    "Футбольная команда выиграла важный матч сезона",
    "Спортсмены готовятся к чемпионату мира по легкой атлетике",
    "Олимпийские игры собирают лучших спортсменов со всего мира",
    "Теннисист выиграл турнир большого шлема",
    "Баскетбольная лига объявила о начале нового сезона"
]

categories = ["ИИ", "ИИ", "ИИ", "ИИ", "ИИ", 
             "Экономика", "Экономика", "Экономика", "Экономика", "Экономика", 
             "Спорт", "Спорт", "Спорт", "Спорт", "Спорт"]

# Создаем датафрейм
df = pd.DataFrame({"text": texts, "category": categories})

# Разделяем на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    df["text"], df["category"], test_size=0.3, random_state=42, stratify=df["category"]
)

# Создаем пайплайн для векторизации и классификации
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1, 2), max_features=1000)),
    ('svm', SVC(kernel='linear', probability=True))
])

# Задаем параметры для поиска по сетке
param_grid = {
    'tfidf__max_features': [500, 1000],
    'tfidf__ngram_range': [(1, 1), (1, 2)],
    'svm__C': [0.1, 1, 10],
    'svm__kernel': ['linear', 'rbf']
}

# Поиск оптимальных параметров с использованием кросс-валидации
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', verbose=1)
grid_search.fit(X_train, y_train)

print(f"Наилучшие параметры: {grid_search.best_params_}")
print(f"Наилучший результат: {grid_search.best_score_:.2f}")

# Оценка на тестовых данных
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
print("\nОтчет по классификации на тестовых данных:")
print(classification_report(y_test, y_pred))

# Визуализация матрицы ошибок
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", 
            xticklabels=grid_search.classes_, 
            yticklabels=grid_search.classes_)
plt.xlabel("Предсказанная категория")
plt.ylabel("Истинная категория")
plt.title("Матрица ошибок")
plt.tight_layout()
plt.show()

# Анализ важных признаков для линейного SVM
if 'linear' in grid_search.best_params_['svm__kernel']:
    # Получаем коэффициенты признаков
    feature_names = best_model.named_steps['tfidf'].get_feature_names_out()
    
    # Визуализация топ-10 признаков для каждого класса
    for i, category in enumerate(grid_search.classes_):
        # Получаем коэффициенты для класса
        if len(grid_search.classes_) == 2:
            # Для бинарной классификации
            coefs = best_model.named_steps['svm'].coef_[0]
        else:
            # Для многоклассовой классификации
            coefs = best_model.named_steps['svm'].coef_[i]
        
        # Сортируем признаки по абсолютным значениям коэффициентов
        top_features_idx = np.argsort(np.abs(coefs))[-10:]
        top_features = [(feature_names[idx], coefs[idx]) for idx in top_features_idx]
        
        # Визуализация
        plt.figure(figsize=(10, 6))
        features, scores = zip(*top_features)
        plt.barh(range(len(features)), scores)
        plt.yticks(range(len(features)), features)
        plt.title(f"Топ-10 признаков для класса '{category}'")
        plt.xlabel("Коэффициент")
        plt.tight_layout()
        plt.show()

In [ ]:
import torch
import numpy as np
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, TensorDataset, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# Создаем набор данных
texts = [
    "Нейронные сети обеспечивают высокую точность распознавания образов",
    "Глубокое обучение произвело революцию в компьютерном зрении",
    "Трансформеры показывают отличные результаты в обработке языка",
    "Алгоритмы машинного обучения применяются для прогнозирования",
    "Искусственный интеллект решает сложные задачи оптимизации",
    "Рекуррентные сети позволяют анализировать последовательные данные",
    "Сверточные сети отлично работают с изображениями и видео",
    "Экономика России показывает признаки восстановления",
    "Инфляция влияет на покупательную способность населения",
    "Центральный банк регулирует финансовую систему страны",
    "Анализ экономических показателей помогает принимать решения",
    "Рынок недвижимости переживает период роста цен",
    "Экономические санкции оказывают влияние на торговые отношения",
    "Биржевые индексы отражают состояние экономики",
    "Футбольная команда выиграла важный матч сезона",
    "Спортсмены готовятся к чемпионату мира по легкой атлетике",
    "Олимпийские игры собирают лучших спортсменов со всего мира",
    "Теннисист выиграл турнир большого шлема",
    "Баскетбольная лига объявила о начале нового сезона",
    "Хоккейная команда провела успешные сборы перед чемпионатом",
    "Марафонец установил новый рекорд на соревнованиях"
]

categories = ["ИИ", "ИИ", "ИИ", "ИИ", "ИИ", "ИИ", "ИИ",
              "Экономика", "Экономика", "Экономика", "Экономика", "Экономика", "Экономика", "Экономика",
              "Спорт", "Спорт", "Спорт", "Спорт", "Спорт", "Спорт", "Спорт"]

# Создаем датафрейм
df = pd.DataFrame({"text": texts, "category": categories})

# Кодируем метки
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['category'])

# Разделяем на обучающую и тестовую выборки
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['category'])

# Загружаем токенизатор и модель BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForSequenceClassification.from_pretrained(
    'bert-base-multilingual-cased',
    num_labels=len(label_encoder.classes_)
)

# Максимальная длина последовательности
MAX_LEN = 128

# Функция для токенизации данных
def tokenize_data(texts, labels, tokenizer, max_len):
    input_ids = []
    attention_masks = []
    
    for text in texts:
        encoded_dict = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])
    
    # Преобразуем списки в тензоры
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(labels)
    
    return input_ids, attention_masks, labels

# Токенизируем обучающие и тестовые данные
train_input_ids, train_attention_masks, train_labels = tokenize_data(
    train_df['text'].values, train_df['label'].values, tokenizer, MAX_LEN
)

test_input_ids, test_attention_masks, test_labels = tokenize_data(
    test_df['text'].values, test_df['label'].values, tokenizer, MAX_LEN
)

# Создаем DataLoader для обучения
batch_size = 4

train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_labels)
train_dataloader = DataLoader(
    train_dataset,
    sampler=RandomSampler(train_dataset),
    batch_size=batch_size
)

test_dataset = TensorDataset(test_input_ids, test_attention_masks, test_labels)
test_dataloader = DataLoader(
    test_dataset,
    sampler=SequentialSampler(test_dataset),
    batch_size=batch_size
)

# Настраиваем оптимизатор
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)

# Количество эпох обучения
epochs = 4

# Общее количество шагов обучения
total_steps = len(train_dataloader) * epochs

# Создаем планировщик скорости обучения
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

# Функция для обучения модели
def train_model(model, dataloader, optimizer, scheduler, device='cpu'):
    model.train()
    total_loss = 0
    
    for batch in tqdm(dataloader):
        batch = tuple(t.to(device) for t in batch)
        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1],
            'labels': batch[2]
        }
        
        # Обнуляем градиенты
        optimizer.zero_grad()
        
        # Прямой проход
        outputs = model(**inputs)
        loss = outputs.loss
        
        # Обратный проход
        loss.backward()
        
        # Ограничиваем градиенты для стабильности
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        # Обновляем веса
        optimizer.step()
        
        # Обновляем скорость обучения
        scheduler.step()
        
        total_loss += loss.item()
    
    return total_loss / len(dataloader)

# Функция для оценки модели
def evaluate_model(model, dataloader, device='cpu'):
    model.eval()
    predictions = []
    true_labels = []
    
    for batch in dataloader:
        batch = tuple(t.to(device) for t in batch)
        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1],
            'labels': batch[2]
        }
        
        with torch.no_grad():
            outputs = model(**inputs)
        
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1).cpu().numpy()
        labels = batch[2].cpu().numpy()
        
        predictions.extend(preds)
        true_labels.extend(labels)
    
    return predictions, true_labels

# Обучаем модель
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    
    # Обучение
    train_loss = train_model(model, train_dataloader, optimizer, scheduler, device)
    print(f"Training loss: {train_loss:.4f}")
    
    # Оценка
    predictions, true_labels = evaluate_model(model, test_dataloader, device)
    
    # Вычисление метрик
    accuracy = (np.array(predictions) == np.array(true_labels)).mean()
    print(f"Accuracy: {accuracy:.4f}")

# Финальная оценка модели
predictions, true_labels = evaluate_model(model, test_dataloader, device)

# Конвертируем обратно в текстовые метки
pred_labels = label_encoder.inverse_transform(predictions)
true_labels_text = label_encoder.inverse_transform(true_labels)

# Отчет по классификации
print("\nОтчет по классификации:")
print(classification_report(true_labels_text, pred_labels))

# Визуализация матрицы ошибок
cm = confusion_matrix(true_labels_text, pred_labels)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", 
            xticklabels=label_encoder.classes_, 
            yticklabels=label_encoder.classes_)
plt.xlabel("Предсказанная категория")
plt.ylabel("Истинная категория")
plt.title("Матрица ошибок")
plt.tight_layout()
plt.show()

# Функция для классификации новых текстов
def classify_text(text, model, tokenizer, label_encoder, device='cpu'):
    encoded_dict = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=MAX_LEN,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    
    input_ids = encoded_dict['input_ids'].to(device)
    attention_mask = encoded_dict['attention_mask'].to(device)
    
    model.eval()
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=1).cpu().numpy()[0]
    predicted_class = np.argmax(probabilities)
    predicted_label = label_encoder.inverse_transform([predicted_class])[0]
    
    return predicted_label, probabilities

# Проверка на новых текстах
new_texts = [
    "Нейросети совершенствуют алгоритмы распознавания лиц",
    "Валютный курс влияет на экспортные операции",
    "Соревнования по плаванию собрали рекордное количество участников"
]

for text in new_texts:
    label, probs = classify_text(text, model, tokenizer, label_encoder, device)
    print(f"\nТекст: {text}")
    print(f"Предсказанная категория: {label}")
    print("Вероятности по классам:")
    for i, cls in enumerate(label_encoder.classes_):
        print(f"  {cls}: {probs[i]:.4f}")

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Простая модель Seq2Seq
class SimpleSeq2Seq(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleSeq2Seq, self).__init__()
        self.hidden_size = hidden_size
        
        # Энкодер
        self.encoder = nn.GRU(input_size, hidden_size)
        
        # Декодер
        self.decoder = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, input_seq, target_seq, teacher_forcing_ratio=0.5):
        input_length = input_seq.size(0)
        target_length = target_seq.size(0)
        batch_size = input_seq.size(1)
        output_size = self.out.out_features
        
        # Тензор для хранения выходов декодера
        outputs = torch.zeros(target_length, batch_size, output_size)
        
        # Прогон через энкодер
        _, hidden = self.encoder(input_seq)
        
        # Первый вход декодера - специальный токен начала предложения
        decoder_input = torch.zeros(1, batch_size, self.hidden_size)
        
        # Поэтапная генерация выходной последовательности
        for t in range(target_length):
            output, hidden = self.decoder(decoder_input, hidden)
            output = self.softmax(self.out(output.squeeze(0)))
            outputs[t] = output
            
            # Решение об использовании teacher forcing
            use_teacher_forcing = True if np.random.random() < teacher_forcing_ratio else False
            
            if use_teacher_forcing:
                # Используем фактический токен из целевой последовательности
                decoder_input = target_seq[t].unsqueeze(0)
            else:
                # Используем предсказанный токен
                topv, topi = output.topk(1)
                decoder_input = topi.detach()
                
        return outputs

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, dropout=0.1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True, bidirectional=True)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input_seq):
        embedded = self.dropout(self.embedding(input_seq))
        output, hidden = self.gru(embedded)
        return output, hidden

class AttentionLayer(nn.Module):
    def __init__(self, hidden_size):
        super(AttentionLayer, self).__init__()
        self.hidden_size = hidden_size
        self.attn = nn.Linear(hidden_size * 3, hidden_size)
        self.v = nn.Parameter(torch.rand(hidden_size))
        
    def forward(self, hidden, encoder_outputs):
        seq_len = encoder_outputs.size(1)
        h = hidden.repeat(seq_len, 1, 1).transpose(0, 1)
        
        energy = torch.tanh(self.attn(torch.cat([h, encoder_outputs], dim=2)))
        energy = energy.transpose(1, 2)
        
        v = self.v.repeat(encoder_outputs.size(0), 1).unsqueeze(1)
        attention = torch.bmm(v, energy).squeeze(1)
        
        return F.softmax(attention, dim=1).unsqueeze(1)

class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout=0.1):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.attention = AttentionLayer(hidden_size)
        self.gru = nn.GRU(hidden_size * 2, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input_token, hidden, encoder_outputs):
        embedded = self.dropout(self.embedding(input_token))
        
        # Вычисление внимания
        attn_weights = self.attention(hidden[-1].unsqueeze(0), encoder_outputs)
        context = torch.bmm(attn_weights, encoder_outputs)
        
        # Объединение контекста и входа
        rnn_input = torch.cat((embedded, context), dim=2)
        
        # Прогон через GRU
        output, hidden = self.gru(rnn_input, hidden)
        output = self.out(output.squeeze(1))
        
        return output, hidden, attn_weights

class Seq2SeqWithAttention(nn.Module):
    def __init__(self, encoder, decoder, device):
        super(Seq2SeqWithAttention, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, source, target, teacher_forcing_ratio=0.5):
        batch_size = source.size(0)
        target_len = target.size(1)
        target_vocab_size = self.decoder.out.out_features
        
        # Инициализация выходов
        outputs = torch.zeros(batch_size, target_len, target_vocab_size).to(self.device)
        
        # Получение выходов энкодера
        encoder_outputs, hidden = self.encoder(source)
        
        # Первый вход декодера
        input_token = target[:, 0]
        
        for t in range(1, target_len):
            output, hidden, _ = self.decoder(input_token.unsqueeze(1), hidden, encoder_outputs)
            outputs[:, t, :] = output
            
            # Решение об использовании teacher forcing
            teacher_force = torch.rand(1).item() < teacher_forcing_ratio
            
            # Выбор следующего входа декодера
            top1 = output.argmax(1)
            input_token = target[:, t] if teacher_force else top1
            
        return outputs

In [ ]:
import torch
from transformers import MarianMTModel, MarianTokenizer
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# Загрузка предобученной модели для перевода с английского на русский
model_name = "Helsinki-NLP/opus-mt-en-ru"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

def translate(texts, model, tokenizer, batch_size=4):
    """
    Функция для перевода текста с использованием модели Transformer
    """
    # Разбиение на батчи для экономии памяти
    results = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        
        # Подготовка текста для модели
        batch_tokenized = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=512)
        
        # Генерация перевода
        translated = model.generate(**batch_tokenized)
        
        # Декодирование результатов
        translated_texts = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
        results.extend(translated_texts)
    
    return results

# Пример использования
english_texts = [
    "Machine learning is a fascinating field.",
    "Neural networks have revolutionized natural language processing.",
    "Transformer architecture provides significant improvements for translation tasks."
]

# Выполнение перевода
russian_translations = translate(english_texts, model, tokenizer)

# Визуализация внимания (можно использовать для анализа работы модели)
def visualize_attention(text, translation, model, tokenizer):
    """
    Визуализация матрицы внимания для понимания работы модели
    """
    # Токенизация текста
    input_ids = tokenizer(text, return_tensors="pt").input_ids
    
    # Получение выходов модели с сохранением внимания
    outputs = model(input_ids, output_attentions=True)
    
    # Получение весов внимания (для последнего слоя и первой головки внимания)
    attention = outputs.attentions[-1][0, 0].detach().numpy()
    
    # Получение токенов для визуализации
    input_tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
    
    # Визуализация матрицы внимания
    plt.figure(figsize=(10, 8))
    plt.imshow(attention, cmap='viridis')
    plt.xticks(range(len(input_tokens)), input_tokens, rotation=90)
    plt.yticks(range(len(input_tokens)), input_tokens)
    plt.colorbar()
    plt.title(f"Attention Matrix for: '{text}'")
    plt.tight_layout()
    plt.show()
    
    # Возврат перевода для сравнения
    return translation

# Анализ внедрений (embeddings) для понимания семантического пространства
def analyze_embeddings(texts, model, tokenizer):
    """
    Анализ и визуализация векторных представлений слов
    """
    embeddings = []
    all_tokens = []
    
    for text in texts:
        inputs = tokenizer(text, return_tensors="pt")
        with torch.no_grad():
            outputs = model.get_encoder()(inputs.input_ids)
        
        # Получение токенов и соответствующих эмбеддингов
        tokens = tokenizer.convert_ids_to_tokens(inputs.input_ids[0])
        text_embeddings = outputs.last_hidden_state[0].numpy()
        
        embeddings.append(text_embeddings)
        all_tokens.extend(tokens)
    
    # Объединение всех эмбеддингов
    all_embeddings = np.vstack([emb for batch_emb in embeddings for emb in batch_emb])
    
    # Снижение размерности для визуализации
    pca = PCA(n_components=2)
    reduced_embeddings = pca.fit_transform(all_embeddings)
    
    # Визуализация
    plt.figure(figsize=(12, 10))
    plt.scatter(reduced_embeddings[:, 0], reduced_embeddings[:, 1], alpha=0.7)
    
    # Добавление меток слов
    for i, token in enumerate(all_tokens):
        plt.annotate(token, (reduced_embeddings[i, 0], reduced_embeddings[i, 1]))
    
    plt.title("Визуализация векторных представлений слов")
    plt.xlabel("Компонента 1")
    plt.ylabel("Компонента 2")
    plt.grid(True)
    plt.show()

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import numpy as np

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length=5000):
        super(PositionalEncoding, self).__init__()
        
        # Создание матрицы позиционного кодирования
        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        
        # Регистрация буфера (постоянного тензора, не являющегося параметром)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        # x: [batch_size, seq_len, d_model]
        return x + self.pe[:, :x.size(1), :]

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads, dropout=0.1):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0
        
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        
        # Линейные преобразования для Q, K, V и выхода
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
        
        self.dropout = nn.Dropout(dropout)
        
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        # Q, K, V: [batch_size, num_heads, seq_len, d_k]
        
        # Вычисление scores между запросами и ключами
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        
        # Применение маски (если она есть)
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)
        
        # Применение softmax для получения весов внимания
        attention_weights = F.softmax(scores, dim=-1)
        attention_weights = self.dropout(attention_weights)
        
        # Взвешивание значений
        output = torch.matmul(attention_weights, V)
        
        return output, attention_weights
    
    def forward(self, Q, K, V, mask=None):
        batch_size = Q.size(0)
        
        # Линейные преобразования и разделение на головки
        q = self.W_q(Q).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        k = self.W_k(K).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        v = self.W_v(V).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        
        # Вычисление внимания
        output, attention = self.scaled_dot_product_attention(q, k, v, mask)
        
        # Объединение головок и финальное линейное преобразование
        output = output.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)
        output = self.W_o(output)
        
        return output, attention

class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(FeedForward, self).__init__()
        
        self.linear1 = nn.Linear(d_model, d_ff)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(d_ff, d_model)
        
    def forward(self, x):
        # x: [batch_size, seq_len, d_model]
        return self.linear2(self.dropout(F.relu(self.linear1(x))))

class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super(EncoderLayer, self).__init__()
        
        self.self_attention = MultiHeadAttention(d_model, num_heads, dropout)
        self.feed_forward = FeedForward(d_model, d_ff, dropout)
        
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        
    def forward(self, x, mask=None):
        # Мультиголовое самовнимание
        attn_output, _ = self.self_attention(x, x, x, mask)
        x = self.norm1(x + self.dropout1(attn_output))
        
        # Полносвязная сеть
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout2(ff_output))
        
        return x

class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super(DecoderLayer, self).__init__()
        
        self.self_attention = MultiHeadAttention(d_model, num_heads, dropout)
        self.cross_attention = MultiHeadAttention(d_model, num_heads, dropout)
        self.feed_forward = FeedForward(d_model, d_ff, dropout)
        
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.dropout3 = nn.Dropout(dropout)
        
    def forward(self, x, enc_output, src_mask=None, tgt_mask=None):
        # Маскированное самовнимание
        attn_output, _ = self.self_attention(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout1(attn_output))
        
        # Кросс-внимание (между декодером и выходом энкодера)
        attn_output, attention_weights = self.cross_attention(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout2(attn_output))
        
        # Полносвязная сеть
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout3(ff_output))
        
        return x, attention_weights

class Encoder(nn.Module):
    def __init__(self, vocab_size, d_model, num_layers, num_heads, d_ff, max_seq_len, dropout=0.1):
        super(Encoder, self).__init__()
        
        self.d_model = d_model
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoding = PositionalEncoding(d_model, max_seq_len)
        
        self.enc_layers = nn.ModuleList([
            EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)
        ])
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask=None):
        # x: [batch_size, seq_len]
        
        # Эмбеддинг и позиционное кодирование
        x = self.embedding(x) * math.sqrt(self.d_model)
        x = self.pos_encoding(x)
        x = self.dropout(x)
        
        # Прохождение через слои энкодера
        for layer in self.enc_layers:
            x = layer(x, mask)
            
        return x

class Decoder(nn.Module):
    def __init__(self, vocab_size, d_model, num_layers, num_heads, d_ff, max_seq_len, dropout=0.1):
        super(Decoder, self).__init__()
        
        self.d_model = d_model
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoding = PositionalEncoding(d_model, max_seq_len)
        
        self.dec_layers = nn.ModuleList([
            DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)
        ])
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, enc_output, src_mask=None, tgt_mask=None):
        # x: [batch_size, seq_len]
        attention_weights = {}
        
        # Эмбеддинг и позиционное кодирование
        x = self.embedding(x) * math.sqrt(self.d_model)
        x = self.pos_encoding(x)
        x = self.dropout(x)
        
        # Прохождение через слои декодера
        for i, layer in enumerate(self.dec_layers):
            x, attention = layer(x, enc_output, src_mask, tgt_mask)
            attention_weights[f'decoder_layer{i+1}'] = attention
            
        return x, attention_weights

class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model=512, num_layers=6, 
                 num_heads=8, d_ff=2048, max_seq_len=5000, dropout=0.1):
        super(Transformer, self).__init__()
        
        self.encoder = Encoder(src_vocab_size, d_model, num_layers, num_heads, d_ff, max_seq_len, dropout)
        self.decoder = Decoder(tgt_vocab_size, d_model, num_layers, num_heads, d_ff, max_seq_len, dropout)
        
        self.final_layer = nn.Linear(d_model, tgt_vocab_size)
        
    def create_masks(self, src, tgt):
        # Создание маски для энкодера
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        
        # Создание маски для декодера
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        
        # Создание маски для предотвращения внимания на будущие токены
        seq_len = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_len, seq_len), diagonal=1)).bool()
        tgt_mask = tgt_mask & nopeak_mask
        
        return src_mask, tgt_mask
        
    def forward(self, src, tgt):
        # src: [batch_size, src_seq_len]
        # tgt: [batch_size, tgt_seq_len]
        
        src_mask, tgt_mask = self.create_masks(src, tgt)
        
        # Кодирование входной последовательности
        enc_output = self.encoder(src, src_mask)
        
        # Декодирование с использованием выхода энкодера
        dec_output, attention_weights = self.decoder(tgt, enc_output, src_mask, tgt_mask)
        
        # Финальное линейное преобразование
        output = self.final_layer(dec_output)
        
        return output, attention_weights

# Пример использования модели
def create_transformer_example():
    # Параметры модели
    src_vocab_size = 5000  # Размер словаря исходного языка
    tgt_vocab_size = 5000  # Размер словаря целевого языка
    d_model = 256  # Размерность модели
    num_layers = 3  # Количество слоев энкодера и декодера
    num_heads = 8  # Количество головок внимания
    d_ff = 512  # Размерность скрытого слоя в feed-forward сети
    
    # Создание модели
    transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_layers, num_heads, d_ff)
    
    # Создание тестовых данных
    src = torch.randint(1, src_vocab_size, (2, 10))  # [batch_size, src_seq_len]
    tgt = torch.randint(1, tgt_vocab_size, (2, 12))  # [batch_size, tgt_seq_len]
    
    # Прямой проход
    output, attention_weights = transformer(src, tgt)
    
    print(f"Input shape: {src.shape}")
    print(f"Target shape: {tgt.shape}")
    print(f"Output shape: {output.shape}")
    print(f"Number of attention maps: {len(attention_weights)}")
    
    return transformer, output, attention_weights

In [ ]:
import torch
from transformers import MarianMTModel, MarianTokenizer
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# Загрузка предобученной модели для перевода с английского на русский
model_name = "Helsinki-NLP/opus-mt-en-ru"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

def translate(texts, model, tokenizer, batch_size=4):
    """
    Функция для перевода текста с использованием модели Transformer
    """
    # Разбиение на батчи для экономии памяти
    results = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        
        # Подготовка текста для модели
        batch_tokenized = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=512)
        
        # Генерация перевода
        translated = model.generate(**batch_tokenized)
        
        # Декодирование результатов
        translated_texts = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
        results.extend(translated_texts)
    
    return results

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import numpy as np

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length=5000):
        super(PositionalEncoding, self).__init__()
        
        # Создание матрицы позиционного кодирования
        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        
        # Регистрация буфера (постоянного тензора, не являющегося параметром)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        # x: [batch_size, seq_len, d_model]
        return x + self.pe[:, :x.size(1), :]

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads, dropout=0.1):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0
        
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        
        # Линейные преобразования для Q, K, V и выхода
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
        
        self.dropout = nn.Dropout(dropout)
        
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        # Q, K, V: [batch_size, num_heads, seq_len, d_k]
        
        # Вычисление scores между запросами и ключами
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        
        # Применение маски (если она есть)
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)
        
        # Применение softmax для получения весов внимания
        attention_weights = F.softmax(scores, dim=-1)
        attention_weights = self.dropout(attention_weights)
        
        # Взвешивание значений
        output = torch.matmul(attention_weights, V)
        
        return output, attention_weights
    
    def forward(self, Q, K, V, mask=None):
        batch_size = Q.size(0)
        
        # Линейные преобразования и разделение на головки
        q = self.W_q(Q).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        k = self.W_k(K).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        v = self.W_v(V).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        
        # Вычисление внимания
        output, attention = self.scaled_dot_product_attention(q, k, v, mask)
        
        # Объединение головок и финальное линейное преобразование
        output = output.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)
        output = self.W_o(output)
        
        return output, attention

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

# Простая модель на основе LSTM для генерации текста
class SimpleLanguageModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers):
        super(SimpleLanguageModel, self).__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)
        
    def forward(self, x, hidden=None):
        embedded = self.embedding(x)
        output, hidden = self.lstm(embedded, hidden)
        output = self.fc(output)
        return output, hidden
    
    def init_hidden(self, batch_size, device):
        h0 = torch.zeros(self.lstm.num_layers, batch_size, self.lstm.hidden_size).to(device)
        c0 = torch.zeros(self.lstm.num_layers, batch_size, self.lstm.hidden_size).to(device)
        return (h0, c0)
    
# Подготовка данных для обучения
def prepare_text_data(text, seq_length):
    words = word_tokenize(text.lower())
    # Создание словаря
    vocab = set(words)
    word_to_idx = {word: i+1 for i, word in enumerate(vocab)}
    word_to_idx['<PAD>'] = 0  # Добавляем токен для padding
    idx_to_word = {i: word for word, i in word_to_idx.items()}
    
    # Преобразование слов в индексы
    indexed_text = [word_to_idx[word] for word in words]
    
    # Создание пар входов и целей
    inputs = []
    targets = []
    for i in range(0, len(indexed_text) - seq_length):
        inputs.append(indexed_text[i:i+seq_length])
        targets.append(indexed_text[i+1:i+seq_length+1])
    
    return inputs, targets, word_to_idx, idx_to_word, len(vocab) + 1

# Генерация текста с обученной моделью
def generate_text(model, seed_text, word_to_idx, idx_to_word, seq_length, num_words, device):
    model.eval()
    words = word_tokenize(seed_text.lower())
    state_h, state_c = model.init_hidden(1, device)
    
    # Преобразование seed_text в индексы
    for word in words:
        if word in word_to_idx:
            x = torch.tensor([[word_to_idx[word]]]).to(device)
            output, (state_h, state_c) = model(x, (state_h, state_c))
    
    # Генерация новых слов
    result = words.copy()
    
    for _ in range(num_words):
        x = torch.tensor([[word_to_idx[words[-1]]]]).to(device)
        output, (state_h, state_c) = model(x, (state_h, state_c))
        
        # Выбор наиболее вероятного слова
        probs = torch.softmax(output[0, -1], dim=0)
        predicted_idx = torch.multinomial(probs, 1).item()
        
        predicted_word = idx_to_word[predicted_idx]
        result.append(predicted_word)
        words.append(predicted_word)
        words = words[1:]  # Скользящее окно
    
    return ' '.join(result)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads, dropout=0.1):
        super(MultiHeadAttention, self).__init__()
        
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        
        self.query = nn.Linear(d_model, d_model)
        self.key = nn.Linear(d_model, d_model)
        self.value = nn.Linear(d_model, d_model)
        self.out = nn.Linear(d_model, d_model)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask=None):
        batch_size = x.size(0)
        
        # Linear projections
        q = self.query(x).view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)
        k = self.key(x).view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)
        v = self.value(x).view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)
        
        # Attention scores
        scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(self.head_dim)
        
        # Apply mask for decoder's self-attention
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)
        
        # Apply softmax
        attention = F.softmax(scores, dim=-1)
        attention = self.dropout(attention)
        
        # Apply attention to values
        out = torch.matmul(attention, v)
        out = out.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)
        out = self.out(out)
        
        return out

class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(FeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        x = self.dropout(F.gelu(self.fc1(x)))
        x = self.fc2(x)
        return x

class DecoderBlock(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super(DecoderBlock, self).__init__()
        
        self.self_attention = MultiHeadAttention(d_model, num_heads, dropout)
        self.feed_forward = FeedForward(d_model, d_ff, dropout)
        
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        
    def forward(self, x, mask=None):
        # Self-attention block
        attn_output = self.self_attention(x, mask)
        x = self.norm1(x + self.dropout1(attn_output))
        
        # Feed forward block
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout2(ff_output))
        
        return x

class GPTModel(nn.Module):
    def __init__(self, vocab_size, d_model, num_layers, num_heads, d_ff, max_seq_len, dropout=0.1):
        super(GPTModel, self).__init__()
        
        self.token_embedding = nn.Embedding(vocab_size, d_model)
        self.position_embedding = nn.Embedding(max_seq_len, d_model)
        
        self.dropout = nn.Dropout(dropout)
        
        self.layers = nn.ModuleList([
            DecoderBlock(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)
        ])
        
        self.final_norm = nn.LayerNorm(d_model)
        self.fc = nn.Linear(d_model, vocab_size)
        
    def get_attention_mask(self, size):
        # Создание маски для предотвращения attention-связей с будущими токенами
        mask = torch.triu(torch.ones(size, size), diagonal=1).bool()
        return ~mask
        
    def forward(self, x):
        batch_size, seq_len = x.size()
        
        # Создание позиционных индексов
        positions = torch.arange(0, seq_len).unsqueeze(0).repeat(batch_size, 1).to(x.device)
        
        # Получение эмбеддингов слов и позиций
        tok_emb = self.token_embedding(x)
        pos_emb = self.position_embedding(positions)
        
        # Сложение эмбеддингов и применение dropout
        x = self.dropout(tok_emb + pos_emb)
        
        # Маска для causal attention (предотвращение внимания на будущие токены)
        attention_mask = self.get_attention_mask(seq_len).to(x.device)
        
        # Проход через слои декодера
        for layer in self.layers:
            x = layer(x, attention_mask)
            
        x = self.final_norm(x)
        # Проекция на словарь
        logits = self.fc(x)
        
        return logits
    
    def generate(self, idx, max_new_tokens, temperature=1.0, top_k=None):
        # idx имеет форму [batch_size, seq_len]
        self.eval()
        
        with torch.no_grad():
            for _ in range(max_new_tokens):
                # Обрезаем контекст, если он стал слишком длинным
                idx_cond = idx if idx.size(1) <= self.position_embedding.num_embeddings else idx[:, -self.position_embedding.num_embeddings:]
                
                # Прямой проход через модель
                logits = self(idx_cond)
                
                # Фокусируемся только на последнем токене
                logits = logits[:, -1, :] / temperature
                
                # Опционально применяем top-k сэмплирование
                if top_k is not None:
                    v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                    logits[logits < v[:, [-1]]] = -float('Inf')
                
                # Применяем softmax для получения вероятностей
                probs = F.softmax(logits, dim=-1)
                
                # Сэмплируем следующий токен
                next_idx = torch.multinomial(probs, num_samples=1)
                
                # Добавляем новый токен к последовательности
                idx = torch.cat((idx, next_idx), dim=1)
                
        return idx